In [1]:
import smtplib
import sys
import pandas as pd
import pandas_gbq as pgbq
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import encoders
# from airflow.models import Variable
from datetime import datetime, timedelta
import os

def send_sanity_check_email():
    # Query the data from BigQuery
    query = """
       SELECT * FROM `shopify-pubsub-project.Data_Warehouse_GoogleAds_Staging.Sanity_check`
    """
    df = pgbq.read_gbq(query, project_id="shopify-pubsub-project")
    df = df.fillna({col: 0 for col in df.select_dtypes(include=['number']).columns})

    # Handle NULL values to prevent errors
    df['Source_max_date'] = pd.to_datetime(df['Source_max_date'], errors='coerce')
    df['Staging_max_date'] = pd.to_datetime(df['Staging_max_date'], errors='coerce')
    df['Date1'] = pd.to_datetime(df['Date1'], errors='coerce')  # Convert to datetime

    # Subtract 3 days
    df['Date1_minus_3'] = df['Date1'] - timedelta(days=3)

    # print(df[['Date1', 'Date1_minus_3']])


    # Apply filtering logic
    filtered_df = df[(df['Source_max_date'] != df['Staging_max_date']) & (df['Staging_max_date'] != df['Latest_Valid_Date']) |
                 (df['Source_max_date'] < (df['Date1']) - timedelta(days=1)) |
                 (df['Source_pk_count'] != df['Staging_pk_count'])]
    # print("The filtered_df is as -",filtered_df.head())
    return filtered_df

In [2]:
new_df = send_sanity_check_email()

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [3]:
new_df

,table_name,source_table,Source_max_date,Staging_max_date,Latest_Valid_Date,Date1,Source_pk_count,Staging_pk_count,Date1_minus_3
